In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cargo DF de train y los mergeo

In [ ]:
train_tran = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TP3-DATOS-2C2021/train_transaction.csv',encoding='Latin-1')
train_id = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TP3-DATOS-2C2021/train_identity.csv',encoding='Latin-1')

In [ ]:
train = train_tran.merge(train_id, on='TransactionID', how='left')

In [ ]:
del train_tran, train_id

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
text_col =[]
def colEsObjet(df,text_col):
  text_col=[x for x in df.columns if df[x].dtype=="object"]
  return text_col

In [ ]:
text_col=colEsObjet(train,text_col)

Saco los NaN reemplazando con 0 y luego creo la veriabels Nan en los features con palabras

In [ ]:
train.fillna(0,inplace=True)

In [ ]:
def crea_nanValue(df,text_col):
    for i in text_col:
        df[i].replace(0,'NaN',inplace=True)

In [ ]:
crea_nanValue(train,text_col)

In [ ]:
enc=OrdinalEncoder()
oneh=OneHotEncoder()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def encodear_CV(df):
  vec=CountVectorizer(max_features=5)
  vec.fit(df.DeviceType)
  fin=vec.transform(df.DeviceType).toarray() # lo paso a array para poder meterlo en un DF
  a=pd.DataFrame(fin,columns=vec.get_feature_names(),index=df.index)
  df=pd.concat([df,a],axis=1)

In [ ]:
def encodear_ohe(df):
  oneh.fit(df[['id_30']])
  fin=oneh.transform(df[['id_30']]).toarray() # lo paso a array para poder meterlo en un DF
  a=pd.DataFrame(fin,columns=oneh.categories_,index=df.index)
  df=pd.concat([df,a],axis=1)

In [ ]:
def encodear_mean(df):
  mean_encoded = df.groupby(['id_31'])['isFraud'].mean().to_dict()
  df['id_31']=df['id_31'].map(mean_encoded)

In [ ]:
def encodear_oren(df,text_col):
  for k in text_col:
    df[text_col]=enc.fit_transform(df[text_col])

In [ ]:
encodear_CV(train)

NameError: ignored

In [ ]:
encodear_ohe(train)

NameError: ignored

In [ ]:
text_col=colEsObjet(train,text_col)

In [ ]:
encodear_oren(train,text_col)

In [ ]:
train.head(5)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,0.0,150.0,discover,142.0,credit,315.0,87.0,19.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,0.0,0.0,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,0.0,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,0.0,0.0,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,0.0,0.0,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,70787.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,0.0,-480.0,New,NotFound,166.0,0.0,542.0,144.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


Genero los encoders que deben usarse:
CountVectorizar con DeviceType
OneHot con card4
Mean encoded con id_31
y el resto con OrdinalEncoder

In [ ]:
vec=CountVectorizer(max_features=5)
vec.fit(train.DeviceType)
fin=vec.transform(train.DeviceType).toarray() # lo paso a array para poder meterlo en un DF
a=pd.DataFrame(fin,columns=vec.get_feature_names(),index=train.index)
train=pd.concat([train,a],axis=1)

In [ ]:
oneh.fit(train[['card4']])
fin=oneh.transform(train[['card4']]).toarray() # lo paso a array para poder meterlo en un DF
a=pd.DataFrame(fin,columns=oneh.categories_,index=train.index)
train=pd.concat([train,a],axis=1)

In [ ]:
mean_encoded = train.groupby(['id_31'])['isFraud'].mean().to_dict()
train['id_31']=train['id_31'].map(mean_encoded)

In [ ]:
train.drop(columns=['DeviceType'],inplace=True)

In [ ]:
text_col=colEsObjet(train,text_col)

In [ ]:
del a, fin

In [ ]:
for k in text_col:
   train[text_col]=enc.fit_transform(train[text_col])

In [ ]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceInfo,desktop,mobile,nan,"(NaN,)","(american express,)","(discover,)","(mastercard,)","(visa,)"
0,2987000,0,86400,68.5,4.0,13926,0.0,150.0,2.0,142.0,2.0,315.0,87.0,19.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,1.0,1.0,41.0,0.021081,0.0,260.0,0.0,1.0,1.0,1.0,1.0,817.0,0,0,1,0.0,0.0,1.0,0.0,0.0
1,2987001,0,86401,29.0,4.0,2755,404.0,150.0,3.0,102.0,2.0,325.0,87.0,0.0,0.0,17.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,1.0,1.0,41.0,0.021081,0.0,260.0,0.0,1.0,1.0,1.0,1.0,817.0,0,0,1,0.0,0.0,0.0,1.0,0.0
2,2987002,0,86469,59.0,4.0,4663,490.0,150.0,4.0,166.0,3.0,330.0,87.0,287.0,0.0,36.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,1.0,1.0,41.0,0.021081,0.0,260.0,0.0,1.0,1.0,1.0,1.0,817.0,0,0,1,0.0,0.0,0.0,0.0,1.0
3,2987003,0,86499,50.0,4.0,18132,567.0,150.0,3.0,117.0,3.0,476.0,87.0,0.0,0.0,54.0,0.0,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,1.0,1.0,41.0,0.021081,0.0,260.0,0.0,1.0,1.0,1.0,1.0,817.0,0,0,1,0.0,0.0,0.0,1.0,0.0
4,2987004,0,86506,50.0,1.0,4497,514.0,150.0,3.0,102.0,2.0,420.0,87.0,0.0,0.0,17.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,100.0,2.0,0.0,-480.0,2.0,2.0,166.0,0.0,542.0,144.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,2.0,2.0,7.0,0.070688,32.0,164.0,4.0,2.0,0.0,2.0,2.0,955.0,0,1,0,0.0,0.0,0.0,1.0,0.0


In [ ]:
Y=train[['isFraud']]

In [ ]:
train.drop(columns=['isFraud'],inplace=True)

Elijo los mejores 120 features que me dice el selector

In [ ]:
select = SelectKBest(f_regression, k=120)

In [ ]:
select.fit(train,Y)

SelectKBest(k=120, score_func=<function f_regression at 0x7fc7d1ba5710>)

In [ ]:
best_cols=select.get_support(indices=True)
best_cols

array([  3,   6,  10,  11,  15,  38,  68,  69,  70,  71,  74,  75,  82,
        84,  85,  86,  87,  90,  91,  92,  93,  95,  96,  97,  98, 103,
       104, 105, 110, 111, 112, 113, 116, 117, 122, 124, 125, 126, 127,
       131, 132, 133, 134, 137, 138, 139, 140, 143, 145, 146, 147, 164,
       165, 166, 176, 178, 200, 223, 224, 229, 237, 238, 239, 241, 242,
       243, 244, 247, 248, 249, 250, 251, 252, 253, 254, 274, 275, 281,
       282, 283, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301,
       302, 303, 304, 305, 308, 309, 310, 311, 312, 313, 314, 315, 335,
       336, 355, 356, 357, 393, 394, 403, 404, 405, 409, 411, 412, 423,
       428, 433, 434])

In [ ]:
best_cols2 = []
for i in range(0,120):
  best_cols2.append(cols[best_cols[i]])

In [ ]:
best_cols2

['ProductCD',
 'card3',
 'addr1',
 'addr2',
 'R_emaildomain',
 'D9',
 'V15',
 'V16',
 'V17',
 'V18',
 'V21',
 'V22',
 'V29',
 'V31',
 'V32',
 'V33',
 'V34',
 'V37',
 'V38',
 'V39',
 'V40',
 'V42',
 'V43',
 'V44',
 'V45',
 'V50',
 'V51',
 'V52',
 'V57',
 'V58',
 'V59',
 'V60',
 'V63',
 'V64',
 'V69',
 'V71',
 'V72',
 'V73',
 'V74',
 'V78',
 'V79',
 'V80',
 'V81',
 'V84',
 'V85',
 'V86',
 'V87',
 'V90',
 'V92',
 'V93',
 'V94',
 'V111',
 'V112',
 'V113',
 'V123',
 'V125',
 'V147',
 'V170',
 'V171',
 'V176',
 'V184',
 'V185',
 'V186',
 'V188',
 'V189',
 'V190',
 'V191',
 'V194',
 'V195',
 'V196',
 'V197',
 'V198',
 'V199',
 'V200',
 'V201',
 'V221',
 'V222',
 'V228',
 'V229',
 'V230',
 'V238',
 'V239',
 'V240',
 'V241',
 'V242',
 'V243',
 'V244',
 'V245',
 'V246',
 'V247',
 'V248',
 'V249',
 'V250',
 'V251',
 'V252',
 'V255',
 'V256',
 'V257',
 'V258',
 'V259',
 'V260',
 'V261',
 'V262',
 'V282',
 'V283',
 'V302',
 'V303',
 'V304',
 'id_01',
 'id_02',
 'id_11',
 'id_12',
 'id_13',
 'id_17'

In [ ]:
X=train[best_cols2]

In [ ]:
X[['id_30','id_31','mobile','desktop','nan']]= train[['id_30','id_31','mobile','desktop','nan']]

In [ ]:
X[['isFraud']]=Y[['isFraud']]

In [ ]:
del train, cols, best_cols,best_cols2

In [ ]:
del Y

In [ ]:
X.head()

,ProductCD,card3,addr1,addr2,R_emaildomain,D9,V15,V16,V17,V18,V21,V22,V29,V31,V32,V33,V34,V37,V38,V39,V40,V42,V43,V44,V45,V50,V51,V52,V57,V58,V59,V60,V63,V64,V69,V71,V72,V73,V74,V78,...,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V255,V256,V257,V258,V259,V260,V261,V262,V282,V283,V302,V303,V304,id_01,id_02,id_11,id_12,id_13,id_17,id_19,id_20,id_31,id_36,mobile,nan,id_30,desktop,isFraud
0,4.0,150.0,315.0,87.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.021081,1.0,0,1,41.0,0,0
1,4.0,150.0,325.0,87.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.021081,1.0,0,1,41.0,0,0
2,4.0,150.0,330.0,87.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.021081,1.0,0,1,41.0,0,0
3,4.0,150.0,476.0,87.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.021081,1.0,0,1,41.0,0,0
4,1.0,150.0,420.0,87.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,70787.0,100.0,2.0,0.0,166.0,542.0,144.0,0.070688,0.0,1,0,7.0,0,0


guardo los DF en CSV para poder usarlos en los modelos de la parteIII

In [ ]:
X.to_csv('/content/drive/MyDrive/Colab Notebooks/TP3-DATOS-2C2021/train_120.csv',index=True)

In [ ]:
train.to_csv('/content/drive/MyDrive/Colab Notebooks/TP3-DATOS-2C2021/train_encodeado.csv',index=True)

In [ ]:
del train

Hago el mismo procedimiento con los DF de Test

In [ ]:
test_tran = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TP3-DATOS-2C2021/test_transaction.csv',encoding='Latin-1')
tes_id = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TP3-DATOS-2C2021/test_identity.csv',encoding='Latin-1')

In [ ]:
test = test_tran.merge(tes_id, on='TransactionID', how='left')

In [ ]:
del test_tran,tes_id

In [ ]:
test.rename(columns={'id-01':'id_01', 'id-02':'id_02', 'id-03':'id_03', 'id-04':'id_04', 'id-05':'id_05',
                        'id-06':'id_06', 'id-07':'id_07', 'id-08':'id_08', 'id-09':'id_09', 'id-10':'id_10',
                        'id-11':'id_11', 'id-12':'id_12', 'id-13':'id_13', 'id-14':'id_14', 'id-15':'id_15',
                        'id-16':'id_16', 'id-17':'id_17', 'id-18':'id_18', 'id-19':'id_19', 'id-20':'id_20',
                        'id-21':'id_21', 'id-22':'id_22', 'id-23':'id_23', 'id-24':'id_24', 'id-25':'id_25',
                        'id-26':'id_26', 'id-27':'id_27', 'id-28':'id_28', 'id-29':'id_29', 'id-30':'id_30',
                        'id-31':'id_31', 'id-32':'id_32', 'id-33':'id_33', 'id-34':'id_34', 'id-35':'id_35',
                        'id-36':'id_36', 'id-37':'id_37', 'id-38':'id_38'},inplace=True)

In [ ]:
text_col=colEsObjet(test,text_col)

In [ ]:
test.fillna(0,inplace=True)

In [ ]:
crea_nanValue(test,text_col)

In [ ]:
fin=vec.transform(test.DeviceType).toarray() # lo paso a array para poder meterlo en un DF
a=pd.DataFrame(fin,columns=vec.get_feature_names(),index=test.index)
test=pd.concat([test,a],axis=1)

In [ ]:
fin=oneh.transform(test[['card4']]).toarray() # lo paso a array para poder meterlo en un DF
a=pd.DataFrame(fin,columns=oneh.categories_,index=test.index)
test=pd.concat([test,a],axis=1)

In [ ]:
test['id_31']=test['id_31'].map(mean_encoded)

In [ ]:
test.drop(columns=['DeviceType'],inplace=True)

In [ ]:
del a, fin

In [ ]:
text_col=colEsObjet(test,text_col)

In [ ]:
for k in text_col:
   test[text_col]=enc.fit_transform(test[text_col])

In [ ]:
test.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceInfo,desktop,mobile,nan,"(NaN,)","(american express,)","(discover,)","(mastercard,)","(visa,)"
0,3663549,18403224,31.95,4.0,10409,111.0,150.0,4.0,226.0,3.0,170.0,87.0,1.0,0.0,17.0,0.0,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,0.0,0.0,0.0,0.0,418.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,1.0,1.0,47.0,0.021081,0.0,390.0,0.0,1.0,1.0,1.0,1.0,1048.0,0,0,1,0.0,0.0,0.0,0.0,1.0
1,3663550,18403263,49.00,4.0,4272,111.0,150.0,4.0,226.0,3.0,299.0,87.0,4.0,0.0,3.0,0.0,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,0.0,0.0,0.0,0.0,231.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,1.0,1.0,47.0,0.021081,0.0,390.0,0.0,1.0,1.0,1.0,1.0,1048.0,0,0,1,0.0,0.0,0.0,0.0,1.0
2,3663551,18403310,171.00,4.0,4476,574.0,150.0,4.0,226.0,3.0,472.0,87.0,2635.0,0.0,20.0,0.0,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,0.0,0.0,0.0,0.0,136.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,1.0,1.0,47.0,0.021081,0.0,390.0,0.0,1.0,1.0,1.0,1.0,1048.0,0,0,1,0.0,0.0,0.0,0.0,1.0
3,3663552,18403310,284.95,4.0,10989,360.0,150.0,4.0,166.0,3.0,205.0,87.0,17.0,0.0,17.0,0.0,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,0.0,0.0,0.0,0.0,242.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,1.0,1.0,47.0,0.021081,0.0,390.0,0.0,1.0,1.0,1.0,1.0,1048.0,0,0,1,0.0,0.0,0.0,0.0,1.0
4,3663553,18403317,67.95,4.0,18018,452.0,150.0,3.0,117.0,3.0,264.0,87.0,6.0,0.0,17.0,0.0,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.0,22.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,22.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,1.0,1.0,47.0,0.021081,0.0,390.0,0.0,1.0,1.0,1.0,1.0,1048.0,0,0,1,0.0,0.0,0.0,1.0,0.0


In [ ]:
test.to_csv('/content/drive/MyDrive/Colab Notebooks/TP3-DATOS-2C2021/test_encodeado.csv',index=True)

In [ ]:
best_features=['ProductCD',
 'card3',
 'addr1',
 'addr2',
 'R_emaildomain',
 'D9',
 'V15',
 'V16',
 'V17',
 'V18',
 'V21',
 'V22',
 'V29',
 'V31',
 'V32',
 'V33',
 'V34',
 'V37',
 'V38',
 'V39',
 'V40',
 'V42',
 'V43',
 'V44',
 'V45',
 'V50',
 'V51',
 'V52',
 'V57',
 'V58',
 'V59',
 'V60',
 'V63',
 'V64',
 'V69',
 'V71',
 'V72',
 'V73',
 'V74',
 'V78',
 'V79',
 'V80',
 'V81',
 'V84',
 'V85',
 'V86',
 'V87',
 'V90',
 'V92',
 'V93',
 'V94',
 'V111',
 'V112',
 'V113',
 'V123',
 'V125',
 'V147',
 'V170',
 'V171',
 'V176',
 'V184',
 'V185',
 'V186',
 'V188',
 'V189',
 'V190',
 'V191',
 'V194',
 'V195',
 'V196',
 'V197',
 'V198',
 'V199',
 'V200',
 'V201',
 'V221',
 'V222',
 'V228',
 'V229',
 'V230',
 'V238',
 'V239',
 'V240',
 'V241',
 'V242',
 'V243',
 'V244',
 'V245',
 'V246',
 'V247',
 'V248',
 'V249',
 'V250',
 'V251',
 'V252',
 'V255',
 'V256',
 'V257',
 'V258',
 'V259',
 'V260',
 'V261',
 'V262',
 'V282',
 'V283',
 'V302',
 'V303',
 'V304',
 'id_01',
 'id_02',
 'id_11',
 'id_12',
 'id_13',
 'id_17',
 'id_19',
 'id_20',
 'id_31',
 'id_36',
 'mobile',
 'nan','id_30','id_31','desktop']

In [ ]:
X=test[best_features]

In [ ]:
X.to_csv('/content/drive/MyDrive/Colab Notebooks/TP3-DATOS-2C2021/test_120.csv',index=True)

In [ ]:
X=test[['TransactionID']]

In [ ]:
X.to_csv('/content/drive/MyDrive/Colab Notebooks/TP3-DATOS-2C2021/test_120_ids.csv',index=True)